# General Functions
-------------------
Common general and useful functions for different notebooks

### Save Keras Model Function

In [5]:
def saveKerasModel(model,model_name,path=None):
    """Save Keras model to disk
    INPUT:
    - model
    - name of the model
    - eventually the path where you want to save
    """
    import onnx
    import keras2onnx
    import os
    
    check = True
    
    #Convert to ONNX
    try:
        onnx_model = keras2onnx.convert_keras(model, model_name)
        print("Type of the model:",type(onnx_model))
        check = True
        
    except:
        print("Impossibile to serialize the model, please check the code and retry")
        check = False
        return check
    
    #Build the path and Save the model on disk
    
    if(path is None):
        path = "..\models"
    else:
        path = path
        
    path_onnx = os.path.join(path,model_name + ".onnx")
    print("Path of the model:",path_onnx)
    
    try:
        #Save model to disk
        onnx.save(onnx_model, path_onnx)
        print("Model",model_name,"saved in:",path_onnx)
        return check
    
    except:
        print("Impossibile to save the model, please check the code and retry")
        check = False
        return check
    


### Load Keras Model Function

In [3]:
def loadKerasModel(path,name=None):
    """Load ONNX Model on Keras
    INPUT:
    - path where the model is
    - eventually the name of the model
    """
    import onnx
    import keras2onnx
    import os
    
    check = True
    
    if(name is None):
        file_path = path
    else:
        name = name + ".onnx"
        file_path = os.path.join(path,name)
        
    onnx_model = onnx.load(file_path)
    
    print("Model loaded:",file_path)
    
    return onnx_model

### Select the best results for a model

In [7]:
def selectBestResults(model1_history,model2_history):
    """Select best results from 2 models
    INPUT:
    - first model history
    - second model history
    """
    model1_acc = max(model1_history.history['val_acc'])
    model1_loss = min(model1_history.history['val_loss'])
    
    model2_acc = max(model1_history.history['val_acc'])
    model2_loss = min(model2_history.history['val_loss'])
     
    print("\nBest results for the model")
    print("Model1 Acc:",model1_acc)
    print("Model1 Loss:",model1_loss)
    print("Model2 Acc:",model2_acc)
    print("Model2 Loss:",model2_loss)
    
    model = 0
    
    #check the best accuracy
    if(model1_acc > model2_acc):
        print("Model 1 best acc")
        model = 1
        return model
    
    elif (model1_acc < model2_acc):
        print("Model 2 best acc")
        model = 2
        return model
    
    #if the two model are equal, check the lowest loss
    else:
        if(model1_loss <= model2_loss):
            print("Model 1 best loss")
            model = 1
            return model
        
        else:
            print("Model 2 best loss")
            model = 2
            return model 
   

In [2]:
def seeErrors(validation_dir,validation_generator,model,display=False):
    """Function that allow to see training errors from model
    INPUTS:
    - validation_dir = directory of the validation dataset
    - validation_generator = generator of images
    - model = model trained
    - display = flag, if you set to True, you can visualize the first 10 errors into your error vector
    WARNING = remember to set the shuffle = false flag inside the validation_generator 
                    if you want to compare the prediction with the ground truth!!!
    """
    from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
    
    # Get the filenames from the generator
    fnames = validation_generator.filenames

    # Get the ground truth from generator
    ground_truth = validation_generator.classes

    # Get the label to class mapping from the generator
    label2index = validation_generator.class_indices

    # Getting the mapping from class index to class label
    idx2label = dict((v,k) for k,v in label2index.items())

    # Get the predictions from the model using the generator
    predictions = model.predict_generator(validation_generator,
                                          steps=validation_generator.samples/validation_generator.batch_size,
                                          verbose=0)
    
    print("\nPrediction completed")
    
    predicted_classes = np.argmax(predictions,axis=1)

    #Check the number of errors
    errors = np.where(predicted_classes != ground_truth)[0]
    print("No of errors = {}/{}".format(len(errors),validation_generator.samples))
    
    #code to display images from the error vector
    if(display is True):
        # Show the errors for the first 10 images inside error vector
        for i in range(len(errors[:10])):
            pred_class = np.argmax(predictions[errors[i]])
            pred_label = idx2label[pred_class]

            title = 'Original label:{}, Prediction :{}, confidence : {:.3f}'.format(
                fnames[errors[i]].split('/')[0],
                pred_label,
                predictions[errors[i]][pred_class])

            original = load_img('{}/{}'.format(validation_dir,fnames[errors[i]]))
            plt.figure(figsize=[7,7])
            plt.axis('off')
            plt.title(title)
            plt.imshow(original)
            plt.show()
    elif(display is False):
        print("Errors not printed")
        
    return errors,predictions

In [6]:
def saveStandardModel(model,model_name,path=None):
    import os
    
    check = True
    
    if(path is None):
        path = "..\models"
    else:
        path = path
        
    file_path = os.path.join(path,model_name + ".h5")
    print("Path of the model:",file_path)
    
    try:
        #Save model to disk
        model.save(file_path)  # creates a HDF5 file 'my_model.h5'
        print("Model",model_name,"saved in:",file_path)
        return check
    
    except:
        print("Impossibile to save the model, please check the code and retry")
        check = False
        return check
        
    
    print("Model saved:",file_path)
    
    return True

In [5]:
def loadStandardModel(path,name=None):
    
    from keras.models import load_model
    
    if(name is None):
        file_path = path
    else:
        name = name + ".h5"
        file_path = os.path.join(path,name)
        
    model = load_model(file_path)  # creates a HDF5 file 'my_model.h5'
    
    print("Model loaded:",file_path)
    
    return model
    

In [1]:
def visualizeImage(directory,img_name,target_size = None):
    """Visualize an image
    INPUT PARAMETERS:
    - directory = directory path where the image is
    - img_name = name of the image (need to specify the extension: jpg, png, ...)
    - targer_size = size of the image do you want to visualize
    """
    import os
    
    #image_path = test_dir + "Apple Braeburn/37_100.jpg"
    
    if(target_size is None):
        target_size = 100
    
    image_path = os.path.join(directory, img_name)
    
    #load the image
    test_image = keras.preprocessing.image.img_to_array(keras.preprocessing.image.load_img(image_path,target_size=(target_size, target_size)))
    
    #rescale the image
    test_image = test_image/255

    plt.imshow(test_image)
    
    return test_image

In [2]:
def confusion_matrix(classes,validation_generator,predictions):
    #Code for generating confusion matrix over the classes
    from sklearn.metrics import classification_report, confusion_matrix
    import seaborn as sn

    print('Creating Confusion Matrix')
    conf_matrix_dataset = confusion_matrix(validation_generator.classes, predictions)

    print('Creating Classification Report')
    classification_report = classification_report(validation_generator.classes, predictions)

    print('Creating final heatmap for the confusion matrix')
    
    df_cm = pd.DataFrame(conf_matrix_dataset, range(classes),
                      range(classes))
    plt.figure(figsize = (classes,classes))
    
    map_confusion = sn.heatmap(df_cm, annot=True,annot_kws={"size": 10})# font size
    
    
    return map_confusion,conf_matrix_dataset,classification_report

#Original code for the confusion matrix
# df_cm = pd.DataFrame(Confusion_matrix, range(103),
#                   range(103))
# plt.figure(figsize = (103,103))
# sn.heatmap(df_cm, annot=True,annot_kws={"size": 10})# font size